# Initial settings

## Packages import

In [ ]:
import explanations # python script
import utils # python script

import os
import plotly.express as px
import dash
from dash import dcc, html, dash_table
import dash_bootstrap_components as dbc
from dash.dependencies import Input, Output, State
from skimage import data
from scipy import ndimage
import cv2
from PIL import Image
from tensorflow.keras.preprocessing import image
import matplotlib.pyplot as plt
import numpy as np
import time

import dash_bootstrap_components as dbc

import base64 # to convert a figure format
from io import BytesIO

from fpdf import FPDF

## Image selection

In [ ]:
IMAGE_NAME = '../images/bird.png'

## Model import

In [ ]:
class Model(object):
    def __init__(self, architecture_utils, architecture, shape, classes = []):
        self.inc_net = architecture_utils
        self.inet_model = architecture
        self.input_shape = shape
        self.target_names = classes # pretrained on ImageNet!


# from tensorflow.keras.applications import resnet_v2 as inc_net
# inet_model = inc_net.ResNet152V2()
# INPUT_SHAPE = (224, 224)

## Choose model

In [ ]:
##### Model pretrained on ImageNet

from tensorflow.keras.applications import inception_v3
model = Model(inception_v3, inception_v3.InceptionV3(), (299, 299)) # pretrained on ImageNet!

##### Model with skin lesions
# from tensorflow.keras.applications import mobilenet_v2
# from tensorflow.keras import models

# target_names = ['melanocytic nevi', 'melanoma', 'basal cell carcinoma', 'Actinic keratoses and intraepithelial carcinoma', 'vascular lesions', 'benign keratosis-like', 'dermatofibroma']
# inet_model = models.load_model('./full_skin_cancer_model.h5')

# model = Model(mobilenet_v2, inet_model, (224, 224), target_names)

## Basic objects -  create image and mask

In [ ]:
n = 500
mask_shape = model.input_shape

img = plt.imread(IMAGE_NAME)
img = image.array_to_img(img)
img = img.resize(mask_shape)
img = image.img_to_array(img)

fig = px.imshow(img, binary_string=True, width=n, height=n)
fig.update_layout(dragmode="drawclosedpath",
                 newshape=dict(fillcolor="cyan", opacity=0.3, line=dict(color="darkblue", width=2)),
                 yaxis_visible=False, yaxis_showticklabels=False,
                 xaxis_visible=False, xaxis_showticklabels=False
                 )

mask = np.zeros(mask_shape)
mask_fig = px.imshow(mask, width=n, height=n)
mask_fig.update_layout(
     yaxis_visible=False, yaxis_showticklabels=False,
     xaxis_visible=False, xaxis_showticklabels=False
)
mask_fig.update_coloraxes(showscale=False) # no bar with color scale

# figure configuration settings
config = {
    "modeBarButtonsToAdd": [
        "drawclosedpath",
        "eraseshape"
    ]
}

# Website design


## Website design for original photo - `objects`

In [ ]:
superpixel_selection = html.Div([
#     html.Div(id='is_disabled', children="False"),
    html.Div([
        html.Div([
            html.Div([
                html.H4("Mark areas in the image", className="my-0 font-weight-normal"),
            ], className="card-header"),
            html.Div([
                dcc.Graph(id="graph-camera", figure=fig, config=config, ),
        
                html.Div(
                    [dcc.Graph(id="graph-mask", figure=mask_fig),],
                    style={"display": "none"},
                ),
    
                ], className="card-body"),
        ], className="card mb-4 box-shadow"),
            
            
        html.Div([
            html.Div([
                html.H4("Options", className="my-0 font-weight-normal"),
            ], className="card-header"),
            
            html.Div([
                ## changes possible


                dcc.Upload(id='upload-image', className="form-control",
                           children=html.Div(['Drag and drop IMAGE or ', html.A('select file')]),
                          ),
                
                dcc.Upload(id='upload-mask', className="form-control",
                           children=html.Div(['Drag and drop MASK or ', html.A('select file')]), 
                          ),

                html.Button("submit uploading", type="button", className="btn btn-lg btn-block btn-outline-primary", id="submit-uploading", n_clicks=0, style={"display": "none"}),          
                
                
                html.Label('Number of inner segments', className="form-label mt-4"),
                dcc.Input(id="inner-segments", placeholder='Number of inner segments...',
                          type='number', value='10', min=0, max=500, className="form-control"),
                html.Label('Number of outer segments', className="form-label mt-4"),
                dcc.Input(id="outer-segments", placeholder='Number of outer segments...',
                          type='number', value='50', min=0, max=500, className="form-control"),
                
                html.Button("Generate results", type="button", className="btn btn-lg btn-block btn-primary", id="submit-val", n_clicks=0, disabled=False),
                html.Button("Enable button", n_clicks=0, type="button", className="btn btn-lg btn-block btn-outline-primary", id="enable-button"),
                
            ], className="card-body"),
        ], className="card mb-4 box-shadow"),
        
    ], className="card-deck mb-3 text-center"),
], className="container")


results_LIMEcraft_and_LIME = html.Div([
    html.Div([            
            
        html.Div([
            html.Div([
                html.H4("LIMEcraft results", className="my-0 font-weight-normal"),
            ], className="card-header"),
            html.Div(id="progress-bar1", children=""),
            
            html.Div([
                html.P(id="result_0"),
                html.Div([html.P(id='details-raport',)], style= {'display': 'none'}),  

                ## LIMEcraft results             
                 dbc.Spinner(children=[dcc.Graph(id="graph-lime", figure=fig),]),

            
            ], className="card-body"),
        ], className="card mb-4 box-shadow"),
        
        
        html.Div([
            html.Div([
                html.H4("LIME results", className="my-0 font-weight-normal"),
            ], className="card-header"),
            html.Div([
               
                 ## LIME results
                 dbc.Spinner(children=[dcc.Graph(id="original-lime", figure=fig)],),
            ], className="card-body"),
        ], className="card mb-4 box-shadow"),
    ], className="card-deck mb-3 text-center"),
], className="container")
 

## Website design for edition

### Color 

In [ ]:
color_edition = html.Div([
    html.Div([
        html.Div([
            html.Div([
                html.H4("Edit image color", className="my-0 font-weight-normal"),
            ], className="card-header"),
            html.Div([
            ##window 1
                dcc.Graph(id="graph-camera-to-edition", figure=fig, config=config, ),
                
                ], className="card-body", style={"margin": 0, "display": "inline-block", "padding": "0 0"}),
        ], className="card mb-4 box-shadow"),
            
        html.Div([
            html.Div([
                html.H4("Options", className="my-0 font-weight-normal"),
            ], className="card-header"),
            
            html.Div([
            ##window 2
                
                html.Label("Red", className="form-label mt-4"),
                dcc.Slider(
                    id='red-slider',
                    min=-255,
                    max=255,
                    step=2,
                    value=0,
                    updatemode='drag',
                    marks={
                        -255: {'label': '-255'},
                        0: {'label': '0'},
                        255: {'label': '255'},
                    },
                            ),
                html.Label("Green", className="form-label mt-4"),
                dcc.Slider(
                    id='green-slider',
                    min=-255,
                    max=255,
                    step=2,
                    value=0,
                    updatemode='drag',
                    marks={
                        -255: {'label': '-255'},
                        0: {'label': '0'},
                        255: {'label': '255'},
                    },
                ),
                html.Label("Blue", className="form-label mt-4"),
                dcc.Slider(
                    id='blue-slider',
                    min=-255,
                    max=255,
                    step=2,
                    value=0,
                    updatemode='drag',
                    marks={
                        -255: {'label': '-255'},
                        0: {'label': '0'},
                        255: {'label': '255'},
                    },
                ),
                html.Button("Change another element", type="button", className="btn btn-lg btn-block btn-outline-primary", id="submit-step"),
                html.Button("Revert edition", type="button", className="btn btn-lg btn-block btn-outline-primary", id="clear-image"),
                html.Button("Generate results", type="button", className="btn btn-lg btn-block btn-primary", id="submit-edition"),
      
            ], className="card-body"),
        ], className="card mb-4 box-shadow", style={"max-width": 250}),
        
        
        html.Div([
            html.Div([
                html.H4("Edition preview", className="my-0 font-weight-normal"),
            ], className="card-header"),
            
            html.Div([
            ##window 3
                html.Div(
                    [dcc.Graph(id="graph-mask-to-edition", figure=mask_fig,)],
                        style={"display": "none",},
                    ),
                
                dcc.Graph(id="graph-camera-edited", figure=fig,),
                
            ], className="card-body", style={"margin": 0, "display": "inline-block", "padding": "0 0"}),
        ], className="card mb-4 box-shadow"),
        
    ], className="card-deck mb-3 text-center"),
], className="container")


color_results = html.Div([
    html.Div([            
        html.Div([
            html.Div([
                html.H4("LIMEcraft results", className="my-0 font-weight-normal"),
            ], className="card-header"),
            
            html.Div([
            ##window4
                
                html.Div(
                    [dcc.Graph(id="graph-mask-edited", figure=mask_fig),],
                    style={"display": "none"},
                ),


                html.Div(id="result_0-edited"),
                html.Div([html.P(id='details-raport-edited',)], style= {'display': 'none'}),

                 dbc.Spinner(children=[dcc.Graph(id="graph-lime-edited", figure=fig),]),
            
            ], className="card-body"),
        ], className="card mb-4 box-shadow"),
        
        
        html.Div([
            html.Div([
                html.H4("Details and comparison of prediction", className="my-0 font-weight-normal"),
            ], className="card-header"),
            html.Div([
            ##window 5
                dash_table.DataTable(
                    id='table',
                    columns=[
                        {'name': 'class', 'id': 'class'},
                        {'name': '%_original_img', 'id': '%_original_img'},
                        {'name': '%_edited_img', 'id': '%_edited_img'}
                    ],
                    data=[],
                ),     


            ], className="card-body"),
        ], className="card mb-4 box-shadow"),
    ], className="card-deck mb-3 text-center"),
], className="container")


### Rotation and location

In [ ]:
move_and_rotate_edition = html.Div([
    html.Div([
        html.Div([
            html.Div([
                html.H4("Move and rotate parts of the image", className="my-0 font-weight-normal"),
            ], className="card-header"),
            html.Div([
            ##window 1
               dcc.Graph(id="graph-camera-to-edition-rotation", figure=fig,config=config),
                
                html.Div(
                    [dcc.Graph(id="graph-mask-to-edition-rotation", figure=mask_fig,)],
                    style={"display": "none",},
                ),

                
                ], className="card-body", style={"margin": 0, "display": "inline-block", "padding": "0 0"}),
        ], className="card mb-4 box-shadow"),
            
        html.Div([
            html.Div([
                html.H4("Options", className="my-0 font-weight-normal"),
            ], className="card-header"),
            
            html.Div([
            ##window 2
              
 
                html.Label('Left(-)/right(+) shifting', className="form-label mt-4"),
                dcc.Input(id="left-right", placeholder='Pixels left/right...',
                          type='number', value='0', className="form-control"),
                
                html.Label('Down(-)/up(+) shifting', className="form-label mt-4"),
                dcc.Input(id="up-down", placeholder='Pixels up/down...',
                          type='number', value='0', className="form-control"),
            
                html.Label('The angle of rotation', className="form-label mt-4"),
                dcc.Input(id="angle-input", placeholder='Enter a value of angle...', type='number', value='0',
                         min=-180, max=180, className="form-control"),
                
                html.Button("Change another element", type="button", className="btn btn-lg btn-block btn-outline-primary", id="submit-step-rotation"),
                html.Button("Revert edition", type="button", className="btn btn-lg btn-block btn-outline-primary", id="clear-image-rotation"),
                html.Button("Generate results", type="button", className="btn btn-lg btn-block btn-primary", id="submit-edition-rotation"),
      
            ], className="card-body"),
        ], className="card mb-4 box-shadow", style={"max-width": 250}),
        
        
        html.Div([
            html.Div([
                html.H4("Edition preview", className="my-0 font-weight-normal"),
            ], className="card-header"),
            
            html.Div([
            ##window 3
              
                dcc.Graph(id="graph-camera-edited-rotation", figure=fig,),

                
            ], className="card-body", style={"margin": 0, "display": "inline-block", "padding": "0 0"}),
        ], className="card mb-4 box-shadow"),
        
    ], className="card-deck mb-3 text-center"),
], className="container")


move_and_rotate_results = html.Div([
    html.Div([            
        html.Div([
            html.Div([
                html.H4("LIMEcraft results", className="my-0 font-weight-normal"),
            ], className="card-header"),
            
            html.Div([
            ##window4
                html.Div(id="result_0-edited-rotation"),
                html.Div([html.P(id='details-raport-edited-rotation',)], style= {'display': 'none'}),
   
                html.Div(
                    [dcc.Graph(id="graph-mask-edited-rotation", figure=mask_fig),],
                    style={"display": "none"},
                ),
               
                dbc.Spinner(children=[dcc.Graph(id="graph-lime-edited-rotation", figure=fig),]),

               
            ], className="card-body"),
        ], className="card mb-4 box-shadow"),
        
        
        html.Div([
            html.Div([
                html.H4("Details and comparison of prediction", className="my-0 font-weight-normal"),
            ], className="card-header"),
            html.Div([
            ##window 5

                dash_table.DataTable(
                    id='table-rotation',
                    columns=[
                        {'name': 'class', 'id': 'class'},
                        {'name': '%_original_img', 'id': '%_original_img'},
                        {'name': '%_edited_img', 'id': '%_edited_img'}
                    ],
                    data=[],
                ),
            

            ], className="card-body"),
        ], className="card mb-4 box-shadow"),
    ], className="card-deck mb-3 text-center"),
], className="container")

### Shape

In [ ]:
shape_edition = html.Div([
    html.Div([
        html.Div([
            html.Div([
                html.H4("Edit image shape", className="my-0 font-weight-normal"),
            ], className="card-header"),
            html.Div([
            ##window 1

                dcc.Graph(id="graph-camera-to-edition-shape", figure=fig, config=config, ),

                html.Div([
                    dcc.Graph(id="graph-mask-to-edition-shape", figure=mask_fig,)],
                        style={"display": "none",}),
                
                ], className="card-body", style={"margin": 0, "display": "inline-block", "padding": "0 0"}),
        ], className="card mb-4 box-shadow"),
            
        html.Div([
            html.Div([
                html.H4("Options", className="my-0 font-weight-normal"),
            ], className="card-header"),
            
            html.Div([
            ##window 2
                html.Label('Power of expansion', className="form-label mt-4"),
                dcc.Input(id="power-input", placeholder='Enter a value of power...', type='number', value='1.4',
                         min=0, max=10, className="form-control"),
                
                html.Button("Change another element", type="button", className="btn btn-lg btn-block btn-outline-primary", id="submit-step-shape"),
                html.Button("Revert edition", type="button", className="btn btn-lg btn-block btn-outline-primary", id="clear-image-shape"),
                html.Button("Generate results", type="button", className="btn btn-lg btn-block btn-primary", id="submit-edition-shape"),     
            ], className="card-body"),
        ], className="card mb-4 box-shadow", style={"max-width": 250}),
        
        
        html.Div([
            html.Div([
                html.H4("Edition preview", className="my-0 font-weight-normal"),
            ], className="card-header"),
            
            html.Div([
            ##window 3
              
                dcc.Graph(id="graph-camera-edited-shape", figure=fig,),
                
            ], className="card-body", style={"margin": 0, "display": "inline-block", "padding": "0 0"}),
        ], className="card mb-4 box-shadow"),
        
    ], className="card-deck mb-3 text-center"),
], className="container")


shape_results = html.Div([
    html.Div([            
        html.Div([
            html.Div([
                html.H4("LIMEcraft results", className="my-0 font-weight-normal"),
            ], className="card-header"),
            
            html.Div([
            ##window4
                html.Div(id="result_0-edited-shape"),
                html.Div([html.P(id='details-raport-edited-shape',)], style= {'display': 'none'}),

                html.Div(
                    [dcc.Graph(id="graph-mask-edited-shape", figure=mask_fig),],
                    style={"display": "none"},
                ),
               
                dbc.Spinner(children=[dcc.Graph(id="graph-lime-edited-shape", figure=fig),]),
            
            
            ], className="card-body"),
        ], className="card mb-4 box-shadow"),
        
        
        html.Div([
            html.Div([
                html.H4("Details and comparison of prediction", className="my-0 font-weight-normal"),
            ], className="card-header"),
            html.Div([
            ##window 5
                
                dash_table.DataTable(
                    id='table-shape',
                    columns=[
                        {'name': 'class', 'id': 'class'},
                        {'name': '%_original_img', 'id': '%_original_img'},
                        {'name': '%_edited_img', 'id': '%_edited_img'}
                    ],
                    data=[],
                ),

            ], className="card-body"),
        ], className="card mb-4 box-shadow"),
    ], className="card-deck mb-3 text-center"),
], className="container")

## Homepage design

In [ ]:
app = dash.Dash(__name__, external_stylesheets=[dbc.themes.BOOTSTRAP])

app.title = "LIMEcraft"

topbar = dbc.Navbar([
    html.Img(className="mb-2", src="https://avatars.githubusercontent.com/u/29011682", alt="", width="24", height="24"),
      html.H5("LIMEcraft", className="my-0 mr-md-auto font-weight-normal"),
      html.Div([
          html.A("Select superpixels", className="p-2 text-dark", href="#sec1"),
          html.A("Edit color", className="p-2 text-dark", href="#sec2"),
          html.A("Move and rotate", className="p-2 text-dark", href="#sec3"),
          html.A("Edit shape", className="p-2 text-dark", href="#sec4")
      ], className="my-2 my-md-0 mr-md-3"),
    html.A("Generate report", className="p-2 text-dark", id="report"),
    html.Div([html.H2('hidden-div', style={'display':'none'}, className="my-0 font-weight-normal text-center",)], id='hidden-div'),

    html.A("About us", className="btn btn-outline-primary", href="https://www.mi2.ai/the-team.html")   
], fixed="top", className="d-flex flex-column flex-md-row align-items-center p-3 px-md-4 mb-3 bg-white border-bottom box-shadow")

header = html.Div(
    [   
        html.Br(),
        html.H1("The LIMEcraft algorithm", className="display-4"),
        html.P("To use the LIMEcraft algorithm, mark at the beginning interesting areas in the image. Then, you can modify the image and compare results of prediction of LIMEcraft algorithm for original and perturbated image.", className="lead"),
    ], className="LIMEcraft-header px-3 py-3 pt-md-5 pb-md-4 mx-auto text-center", style={"max-width": 700}, id="sec1")

content = html.Div(
    [  
        header,
        html.Div(
            [dcc.Graph(id="img_base", figure=fig),],
            style={"display": "none"},
        ),
        
        superpixel_selection,
        results_LIMEcraft_and_LIME,
        
        html.Div([html.H2("Edit color of the image", className="display-4"),], className="my-0 font-weight-normal text-center",id="sec2"),
        color_edition,
        color_results,
        
        html.Div([html.H2("Move, rotate or delete parts of the image", className="display-4"),], className="my-0 font-weight-normal text-center", id="sec3"),
        move_and_rotate_edition,
        move_and_rotate_results,

        html.Div([html.H2("Change shape of the image", className="display-4"),], className="my-0 font-weight-normal text-center", id="sec4"),
        shape_edition,
        shape_results,
        
    ])

app.layout = html.Div([dcc.Location(id="url"), topbar, content])


# Functionalities

## Load image

In [ ]:
@app.callback(
    Output('img_base', 'figure'),
    Input('upload-image', 'contents'),
    prevent_initial_call=True,
)
def update_output(contents):
    if contents is not None:
        img_str = contents.split(",")
        img_ = Image.open(BytesIO(base64.b64decode(str(img_str[1]))))
        img_ = np.array(img_)
        img_ = image.array_to_img(img_)
        img_ = img_.resize(model.input_shape)
        img = image.img_to_array(img_)
        img = img[...,:3] # model does not take image with 4 layers
        
        fig = px.imshow(img, binary_string=True, width=n, height=n)
        fig.update_layout(dragmode="drawclosedpath",
                         newshape=dict(fillcolor="cyan", opacity=0.3, line=dict(color="darkblue", width=2)),
                         yaxis_visible=False, yaxis_showticklabels=False,
                         xaxis_visible=False, xaxis_showticklabels=False,
                          margin=dict(l=0, r=0, t=30, b=10)
                         )
        return fig
    else:
        dash.no_update

@app.callback(
    Output('graph-camera', 'figure'),

    Input('img_base', 'figure'),
)
def update(figure):
    return figure

## LIMEcraft for original photo

In [ ]:
# select areas
@app.callback(
    Output("graph-mask", "figure"),
    
    Input("graph-camera", "relayoutData"),
    Input('upload-mask', 'contents'),
    
    prevent_initial_call=True,
)
def on_new_annotation(relayout_data, contents):
    if contents is not None: # mask upload
        img_str = contents.split(",")
        img_ = Image.open(BytesIO(base64.b64decode(str(img_str[1]))))
        img_ = np.array(img_)
        # in case of loading a picture of a different size
        img_ = image.array_to_img(img_)
        img_ = img_.resize(model.input_shape)
        img_ = image.img_to_array(img_)
        
        img_ = img_[:,:,1]
      
        fig = px.imshow(img_, binary_string=True)
        fig.update_layout(dragmode="drawclosedpath",
                         newshape=dict(fillcolor="cyan", opacity=0.3, line=dict(color="darkblue", width=2)),
                         yaxis_visible=False, yaxis_showticklabels=False,
                         xaxis_visible=False, xaxis_showticklabels=False,
                         )
        return fig
    if "shapes" in relayout_data:
        last_shape = relayout_data["shapes"][-1]
        mask = utils.path_to_mask(last_shape["path"], model.input_shape, relayout_data)

        mask_fig = px.imshow(mask)
        mask_fig.update_layout(
             yaxis_visible=False, yaxis_showticklabels=False,
             xaxis_visible=False, xaxis_showticklabels=False
        )
        mask_fig.update_coloraxes(showscale=False)

        return mask_fig
    else:
        return dash.no_update
    
# approval button for selected areas
@app.callback(
    Output("graph-lime", "figure"),
    Output("result_0", "children"),
    Output("details-raport", "children"),
    Output("graph-mask-edited", "figure"),
    Output("graph-mask-edited-rotation", "figure"),
    Output("graph-mask-edited-shape", "figure"),
    
    Output("original-lime", "figure"),

    Input('submit-val', 'n_clicks'),
    Input("graph-mask", "figure"),
    Input('graph-camera', 'figure'),
    
    Input('img_base', 'figure'),
    Input('submit-uploading', 'n_clicks'),
    
    Input("inner-segments", "value"),
    Input("outer-segments", "value"),
    
    prevent_initial_call=True,
)
def on_submitted_button(n_clicks, figure, img_fig, fig_base, n_clicks2, inner_n_segments, outer_n_segments):
    changed_id = [p['prop_id'] for p in dash.callback_context.triggered][0]

    if 'submit-uploading' in changed_id:
        return [fig_base, _, _, _, _, _, fig_base]
    
    if 'submit-val' in changed_id:
            
        if figure["data"][0].get("z") is not None:
            mask = np.array(figure["data"][0]["z"]) # extract array from figure
        else:
            mask_str = figure["data"][0]["source"]
            mask_str = mask_str.split(",")
            mask = Image.open(BytesIO(base64.b64decode(str(mask_str[1]))))
            mask = np.array(mask)

            mask[mask!=0] = 255
            
        
        img_str = img_fig["data"][0]["source"]
        img_str = img_str.split(",")
        img_ = Image.open(BytesIO(base64.b64decode(str(img_str[1]))))
        img_ = np.array(img_)
        
        
        [img_with_LIMEcraft, preds] = explanations.test_limecraft(model, img_, mask, int(inner_n_segments), int(outer_n_segments))
        img_with_lime = explanations.test_lime(model, img_)

        
        
        img_with_LIMEcraft_fig = px.imshow(img_with_LIMEcraft)
        img_with_LIMEcraft_fig.update_layout(
             yaxis_visible=False, yaxis_showticklabels=False,
             xaxis_visible=False, xaxis_showticklabels=False,
            margin=dict(l=0, r=0, t=30, b=10)
             )
        img_with_lime_fig = px.imshow(img_with_lime)
        img_with_lime_fig.update_layout(
             yaxis_visible=False, yaxis_showticklabels=False,
             xaxis_visible=False, xaxis_showticklabels=False,
            margin=dict(l=0, r=0, t=30, b=10)
             )
        
        return [img_with_LIMEcraft_fig, 
                'This is a/an "{}" with probability {}%'.format(preds[0][1], np.round(np.double(preds[0][2])*100,2)),
                str(preds),
                px.imshow(mask),
                px.imshow(mask),
               px.imshow(mask),
               img_with_lime_fig]
        
    else:
        return dash.no_update

### Edition - `color`

In [ ]:
# create a mask from the selected areas
@app.callback(
    Output("graph-mask-to-edition", "figure"),
    Input("graph-camera-to-edition", "relayoutData"),
    prevent_initial_call=True,
)
def on_new_annotation_color(relayout_data):
    if "shapes" in relayout_data: 
        last_shape = relayout_data["shapes"][-1]
        mask = utils.path_to_mask(last_shape["path"], model.input_shape, relayout_data)
        
        mask_fig = px.imshow(mask, width=n, height=n)
        mask_fig.update_layout(
             yaxis_visible=False, yaxis_showticklabels=False,
             xaxis_visible=False, xaxis_showticklabels=False,
             )
        
        return mask_fig
    else:
        return dash.no_update

    
# change color (buttons)
@app.callback(
    Output("graph-camera-edited", "figure"),
    
    Input("graph-camera-to-edition", "figure"),
    Input("graph-mask-to-edition", "figure"),
    [Input("red-slider", "value"),
    Input("green-slider", "value"),
    Input("blue-slider", "value"),],
    
    Input('img_base', 'figure'),
    Input('submit-uploading', 'n_clicks'),
    
)
def change_color(img_fig, mask_fig, red_val, green_val, blue_val, fig_base, n_clicks): 
    changed_id = [p['prop_id'] for p in dash.callback_context.triggered][0]

    if 'submit-uploading' in changed_id:
        return fig_base
    if "red-slider" in changed_id or "green-slider" in changed_id or "blue-slider" in changed_id:
        img_str = img_fig["data"][0]["source"]
        img_str = img_str.split(",")
        img_ = Image.open(BytesIO(base64.b64decode(str(img_str[1]))))
        img_ = np.array(img_)

        mask_ = np.array(mask_fig["data"][0]["z"]) # extract array from figure

        def create_new_array(array_2d): #convert limited value format of image to normal array
            new_array = np.full(array_2d.shape, None)
            for i in range(array_2d.shape[0]):
                for j in range(array_2d.shape[1]):
                    new_array[i][j] = array_2d[i][j]
            return new_array
        
        # create an auxiliary image, change its color, apply a mask to it, and overlay it on top of the original
        img_colored = create_new_array(img_)
        img_colored[:,:,0] = np.add(img_colored[:,:,0], red_val, casting="unsafe")      
        img_colored[:,:,1] = np.add(img_colored[:,:,1], green_val, casting="unsafe")     
        img_colored[:,:,2] = np.add(img_colored[:,:,2], blue_val, casting="unsafe")

        img_colored = np.clip(img_colored, 0, 255) # in case value not in [0, 255]

        # the mask must be black and white
        mask_[mask_!=0] = 255

        # convert to PIL Image and put 2 images and a mask together
        img_colored = Image.fromarray(np.uint8(img_colored))
        mask_ = Image.fromarray(np.uint8(mask_))
        img_ = Image.fromarray(np.uint8(img_))
        img_.paste(img_colored, mask = mask_)
        
        img__fig = px.imshow(img_, width=n, height=n)
        img__fig.update_layout(
             yaxis_visible=False, yaxis_showticklabels=False,
             xaxis_visible=False, xaxis_showticklabels=False,
            margin=dict(l=0, r=0, t=30, b=10),
             )

        return img__fig
    else:
        return dash.no_update
    

# save edition
@app.callback(
    Output("graph-camera-to-edition", "figure"), # This allows you to select different areas independently of each other and do further editing
    [Output("red-slider", "value"),
    Output("green-slider", "value"),
    Output("blue-slider", "value"),],
    
    Input("submit-step", "n_clicks"),
    Input("graph-camera-edited", "figure"),
    Input("clear-image", "n_clicks"),
    
    Input('img_base', 'figure'),
    Input('submit-uploading', 'n_clicks'),
    
    prevent_initial_call=True,
)
def submit_image_edition(n_clicks, figure, n_clicks2, fig_base, n_clicks3):
    changed_id = [p['prop_id'] for p in dash.callback_context.triggered][0]

    if 'clear-image' in changed_id or 'submit-uploading' in changed_id:
        return [fig_base, 0, 0, 0]
        
    if 'submit-step' in changed_id:
        img_str = figure["data"][0]["source"]
        img_str = img_str.split(",")
        img = Image.open(BytesIO(base64.b64decode(img_str[1])))
        img = np.array(img)
        img_figure = px.imshow(img, width=n, height=n)
        img_figure.update_layout(dragmode="drawclosedpath",
                 newshape=dict(fillcolor="cyan", opacity=0.3, line=dict(color="darkblue", width=2)),
                 yaxis_visible=False, yaxis_showticklabels=False,
                 xaxis_visible=False, xaxis_showticklabels=False,
                 margin=dict(l=0, r=0, t=30, b=10),
                 )
        return [img_figure, 0, 0, 0]
    else:
        return dash.no_update

### Edition - `rotation and shift`

In [ ]:
# create a mask from the selected areas
@app.callback(
    Output("graph-mask-to-edition-rotation", "figure"),
    Input("graph-camera-to-edition-rotation", "relayoutData"),
    prevent_initial_call=True,
)
def on_new_annotation_color(relayout_data):
    if "shapes" in relayout_data: 
        last_shape = relayout_data["shapes"][-1]
        mask = utils.path_to_mask(last_shape["path"], model.input_shape, relayout_data)
        
        mask_fig = px.imshow(mask, width=n, height=n)
        mask_fig.update_layout(
             yaxis_visible=False, yaxis_showticklabels=False,
             xaxis_visible=False, xaxis_showticklabels=False,
             )
        
        return mask_fig
    else:
        return dash.no_update
    
# change shape (buttons)
@app.callback(
    Output("graph-camera-edited-rotation", "figure"),
    
    Input("graph-camera-to-edition-rotation", "relayoutData"),
    Input("graph-camera-to-edition-rotation", "figure"),
    Input("graph-mask-to-edition-rotation", "figure"),
    [Input("angle-input", "value"),    
    Input("left-right", "value"),
    Input("up-down", "value"),],
    
    Input('img_base', 'figure'),
    Input('submit-uploading', 'n_clicks'),
    
)
def change_shape(relayout_data, img_fig, mask_fig, degree, left_right, up_down, fig_base, n_clicks): 
    changed_id = [p['prop_id'] for p in dash.callback_context.triggered][0]

    if 'submit-uploading' in changed_id:
        return fig_base
    if degree is None:
        degree = 0
    
    if relayout_data is not None and "shapes" in relayout_data:
        if len(relayout_data["shapes"]) > 1:
            del relayout_data["shapes"][-1] # rotate only one area
            
        last_shape = relayout_data["shapes"][-1]
        mask = utils.path_to_mask(last_shape["path"], model.input_shape, relayout_data)
        
        mask_fig = px.imshow(mask)
        mask_fig.update_layout(
             yaxis_visible=False, yaxis_showticklabels=False,
             xaxis_visible=False, xaxis_showticklabels=False,
             )
        
        
        # look for the center of gravity of the selected area
        (x_centrum, y_centrum) = utils.find_centrum(relayout_data["shapes"][0]["path"])
        
        points = utils.path_to_indices(relayout_data["shapes"][0]["path"])
        x = [p[0] for p in points]
        y = [p[1] for p in points]
        
        # look for protruding points to draw a rectangle that covers the entire area
        x_min = np.min(x)
        x_max = np.max(x)
        y_min = np.min(y)
        y_max = np.max(y)
        
        radius = np.max([x_centrum-x_min, x_max-x_centrum, 
                        y_centrum-y_min, y_max-y_centrum])
  
        sub_mask = mask[max(y_centrum-radius,0):min(y_centrum+radius+1,model.input_shape[1]), max(x_centrum-radius,0):min(x_centrum+radius+1,model.input_shape[0])]
    
        img_str = img_fig["data"][0]["source"]
        img_str = img_str.split(",")
        img_ = Image.open(BytesIO(base64.b64decode(img_str[1])))
        img_rotated = utils.circle_rotate(img_, x_centrum, y_centrum, radius, int(degree), model.input_shape, mask, sub_mask,
                                    left_right=int(left_right), up_down=int(up_down))
        
        fig_rotated = px.imshow(img_rotated)
        fig_rotated.update_layout(
             yaxis_visible=False, yaxis_showticklabels=False,
             xaxis_visible=False, xaxis_showticklabels=False,
            margin=dict(l=0, r=0, t=30, b=10),
             )
        
        
        return fig_rotated
    
    else:
        return dash.no_update
    
# save edition
@app.callback(
    Output("graph-camera-to-edition-rotation", "figure"), # This allows you to select different areas independently of each other and do further editing
    [Output("angle-input", "value"),    
    Output("left-right", "value"),
    Output("up-down", "value"),],
    
    Input("submit-step-rotation", "n_clicks"),
    Input("graph-camera-edited-rotation", "figure"),
    Input("clear-image-rotation", "n_clicks"),
    
    Input('img_base', 'figure'),
    Input('submit-uploading', 'n_clicks'),
    
    prevent_initial_call=True,
)
def submit_image_edition(n_clicks, figure, n_clicks2, fig_base, n_clicks3):
    changed_id = [p['prop_id'] for p in dash.callback_context.triggered][0]

    if 'clear-image' in changed_id or 'submit-uploading' in changed_id:
        return [fig_base, 0, 0, 0]
        
    if 'submit-step-rotation' in changed_id:
        img_str = figure["data"][0]["source"]
        img_str = img_str.split(",")
        img = Image.open(BytesIO(base64.b64decode(img_str[1])))
        img = np.array(img)
        img_figure = px.imshow(img, width=n, height=n)
        img_figure.update_layout(dragmode="drawclosedpath",
                 newshape=dict(fillcolor="cyan", opacity=0.3, line=dict(color="darkblue", width=2)),
                 yaxis_visible=False, yaxis_showticklabels=False,
                 xaxis_visible=False, xaxis_showticklabels=False,
                 margin=dict(l=0, r=0, t=30, b=10),
                 )
        return [img_figure, 0, 0, 0]
    else:
        return dash.no_update

### Edition - `shape`

In [ ]:
# create a mask from the selected areas
@app.callback(
    Output("graph-mask-to-edition-shape", "figure"),
    Input("graph-camera-to-edition-shape", "relayoutData"),
    prevent_initial_call=True,
)
def on_new_annotation_color(relayout_data):
    if "shapes" in relayout_data: 
        last_shape = relayout_data["shapes"][-1]
        mask = utils.path_to_mask(last_shape["path"], model.input_shape, relayout_data)
        
        mask_fig = px.imshow(mask, width=n, height=n)
        mask_fig.update_layout(
             yaxis_visible=False, yaxis_showticklabels=False,
             xaxis_visible=False, xaxis_showticklabels=False,
             )
        
        return mask_fig
    else:
        return dash.no_update

    
# change color (buttons)
@app.callback(
    Output("graph-camera-edited-shape", "figure"),
    
    Input("graph-camera-to-edition-shape", "relayoutData"),
    Input("graph-camera-to-edition-shape", "figure"),
    Input("graph-mask-to-edition-shape", "figure"),
    Input("power-input", "value"),
    
    Input('img_base', 'figure'),
    Input('submit-uploading', 'n_clicks'),
    
)
def change_color(relayout_data, img_fig, mask_fig, power, fig_base, n_clicks): 
    changed_id = [p['prop_id'] for p in dash.callback_context.triggered][0]

    if 'submit-uploading' in changed_id:
        return fig_base
    if power is None:
        power=1
        
    if relayout_data is not None and "shapes" in relayout_data:
        if len(relayout_data["shapes"]) > 1:
            del relayout_data["shapes"][-1] # rotate only one area
            
        last_shape = relayout_data["shapes"][-1]
        mask = utils.path_to_mask(last_shape["path"], img.shape, relayout_data)
        
        mask_fig = px.imshow(mask)
        mask_fig.update_layout(
             yaxis_visible=False, yaxis_showticklabels=False,
             xaxis_visible=False, xaxis_showticklabels=False,
             )
        
        # look for the center of gravity of the selected area
        (x_centrum, y_centrum) = utils.find_centrum(relayout_data["shapes"][0]["path"])

        
        img_str = img_fig["data"][0]["source"]
        img_str = img_str.split(",")
        img_ = Image.open(BytesIO(base64.b64decode(img_str[1])))
        img_reshaped = utils.remapping(img_, relayout_data["shapes"][0]["path"], float(power))
        
        fig_reshaped = px.imshow(img_reshaped)
        fig_reshaped.update_layout(
             yaxis_visible=False, yaxis_showticklabels=False,
             xaxis_visible=False, xaxis_showticklabels=False,
            margin=dict(l=0, r=0, t=30, b=10),
             )
        
        
        return fig_reshaped
    else:
        return dash.no_update
    

# save edition
@app.callback(
    Output("graph-camera-to-edition-shape", "figure"), # This allows you to select different areas independently of each other and do further editing
    Output("power-input", "value"),
    
    Input("submit-step-shape", "n_clicks"),
    Input("graph-camera-edited-shape", "figure"),
    Input("clear-image-shape", "n_clicks"),
    
    Input('img_base', 'figure'),
    Input('submit-uploading', 'n_clicks'),
    
    prevent_initial_call=True,
)
def submit_image_edition(n_clicks, figure, n_clicks2, fig_base, n_clicks3):
    changed_id = [p['prop_id'] for p in dash.callback_context.triggered][0]

    if 'clear-image' in changed_id or 'submit-uploading' in changed_id:
        return [fig_base, 1]
        
    if 'submit-step-shape' in changed_id:
        img_str = figure["data"][0]["source"]
        img_str = img_str.split(",")
        img = Image.open(BytesIO(base64.b64decode(img_str[1])))
        img = np.array(img)
        img_figure = px.imshow(img, width=n, height=n)
        img_figure.update_layout(dragmode="drawclosedpath",
                 newshape=dict(fillcolor="cyan", opacity=0.3, line=dict(color="darkblue", width=2)),
                 yaxis_visible=False, yaxis_showticklabels=False,
                 xaxis_visible=False, xaxis_showticklabels=False,
                 margin=dict(l=0, r=0, t=30, b=10),
                 )
        return [img_figure, 1]
    else:
        return dash.no_update

## LIMEcraft for edited photo

In [ ]:
# approval button - color
@app.callback(
    Output("graph-lime-edited", "figure"),
    Output("result_0-edited", "children"),
    Output("details-raport-edited", "children"),

    
    Input('submit-edition', 'n_clicks'),
    Input("graph-mask-edited", "figure"),
    Input("graph-camera-edited", "figure"),
    
    Input('img_base', 'figure'),
    Input('submit-uploading', 'n_clicks'),
    
    Input("inner-segments", "value"),
    Input("outer-segments", "value"),
    
    prevent_initial_call=True,
)

def on_submitted_button_edition(n_clicks, mask_fig_, img_fig_, fig_base, n_clicks2, inner_n_segments, outer_n_segments):
    changed_id = [p['prop_id'] for p in dash.callback_context.triggered][0]

    if 'submit-uploading' in changed_id:
        return [fig_base, _, _]
    if 'submit-edition' in changed_id:
        img_str = img_fig_["data"][0]["source"]
        img_str = img_str.split(",")
        img_ = Image.open(BytesIO(base64.b64decode(img_str[1])))
        img_ = np.array(img_)
        
        mask = np.array(mask_fig_["data"][0]["z"]) # extract array from figure
        
        [img_with_LIMEcraft, preds] = explanations.test_limecraft(model, img_, mask, int(inner_n_segments), int(outer_n_segments))
        
        lime_fig = px.imshow(img_with_LIMEcraft)
        lime_fig.update_layout(
             yaxis_visible=False, yaxis_showticklabels=False,
             xaxis_visible=False, xaxis_showticklabels=False,
            margin=dict(l=0, r=0, t=30, b=10)
             )
        
        return [lime_fig, 
                'This is a/an "{}" with probability {}%'.format(preds[0][1], np.round(np.double(preds[0][2])*100,2)),
               str(preds)]  
    else:
        return dash.no_update
    
# approval button - rotation and shift
@app.callback(
    Output("graph-lime-edited-rotation", "figure"),
    Output("result_0-edited-rotation", "children"),
    Output("details-raport-edited-rotation", "children"),
 
    Input('submit-edition-rotation', 'n_clicks'),
    Input("graph-mask-edited-rotation", "figure"),
    Input("graph-camera-edited-rotation", "figure"),
    
    Input('img_base', 'figure'),
    Input('submit-uploading', 'n_clicks'),
    
    Input("inner-segments", "value"),
    Input("outer-segments", "value"),
    
    prevent_initial_call=True,
)

def on_submitted_button_edition_rotation(n_clicks, mask_fig_, img_fig_, fig_base, n_clicks2, inner_n_segments, outer_n_segments):
    changed_id = [p['prop_id'] for p in dash.callback_context.triggered][0]

    if 'submit-uploading' in changed_id:
        return [fig_base, _, _]
    if 'submit-edition-rotation' in changed_id:
        img_str = img_fig_["data"][0]["source"]
        img_str = img_str.split(",")
        img = Image.open(BytesIO(base64.b64decode(img_str[1])))
        img = np.array(img)
        
        mask = np.array(mask_fig_["data"][0]["z"]) # extract array from figure
        
        [img_with_LIMEcraft, preds] = explanations.test_limecraft(model, img, mask, int(inner_n_segments), int(outer_n_segments))
        
        lime_fig = px.imshow(img_with_LIMEcraft)
        lime_fig.update_layout(
             yaxis_visible=False, yaxis_showticklabels=False,
             xaxis_visible=False, xaxis_showticklabels=False,
            margin=dict(l=0, r=0, t=30, b=10)
             )
        return [lime_fig, 
                'This is a/an "{}" with probability {}%'.format(preds[0][1], np.round(np.double(preds[0][2])*100,2)),
               str(preds)]  
    else:
        return dash.no_update
    
# approval button - shape
@app.callback(
    Output("graph-lime-edited-shape", "figure"),
    Output("result_0-edited-shape", "children"),
    Output("details-raport-edited-shape", "children"),
 
    Input('submit-edition-shape', 'n_clicks'),
    Input("graph-mask-edited-shape", "figure"),
    Input("graph-camera-edited-shape", "figure"),
    
    Input('img_base', 'figure'),
    Input('submit-uploading', 'n_clicks'),
    
    Input("inner-segments", "value"),
    Input("outer-segments", "value"),
    
    prevent_initial_call=True,
)

def on_submitted_button_edition_shape(n_clicks, mask_fig_, img_fig_, fig_base, n_clicks2, inner_n_segments, outer_n_segments):
    changed_id = [p['prop_id'] for p in dash.callback_context.triggered][0]

    if 'submit-uploading' in changed_id:
        return [fig_base, _, _]
    if 'submit-edition-shape' in changed_id:
        img_str = img_fig_["data"][0]["source"]
        img_str = img_str.split(",")
        img = Image.open(BytesIO(base64.b64decode(img_str[1])))
        img = np.array(img)
        
        mask = np.array(mask_fig_["data"][0]["z"]) # extract array from figure
        
        [img_with_LIMEcraft, preds] = explanations.test_limecraft(model, img, mask, int(inner_n_segments), int(outer_n_segments))
        
        lime_fig = px.imshow(img_with_LIMEcraft)
        lime_fig.update_layout(
             yaxis_visible=False, yaxis_showticklabels=False,
             xaxis_visible=False, xaxis_showticklabels=False,
            margin=dict(l=0, r=0, t=30, b=10)
             )
        return [lime_fig, 
                'This is a/an "{}" with probability {}%'.format(preds[0][1], np.round(np.double(preds[0][2])*100,2)),
               str(preds)]  
    else:
        return dash.no_update

## Reports

In [ ]:
# frame update - color
@app.callback(
    Output(component_id='table', component_property='data'),
    
    Input('submit-edition', 'n_clicks'),
    Input("details-raport", "children"),
    Input("details-raport-edited", "children")
)
def create_table(n_clicks, str1, str2):
    changed_id = [p['prop_id'] for p in dash.callback_context.triggered][0] 
    
    if 'submit-edition' in changed_id:
        df = utils.merge_df_str(str1, str2)
        return df.to_dict(orient='records')        
    else:
        return dash.no_update
    
# frame update - rotation and shift
@app.callback(
    Output(component_id='table-rotation', component_property='data'),
    
    Input('submit-edition-rotation', 'n_clicks'),
    Input("details-raport", "children"),
    Input("details-raport-edited-rotation", "children"),
    prevent_initial_call=True,
)
def create_table_rotation(n_clicks, str1, str2):
    changed_id = [p['prop_id'] for p in dash.callback_context.triggered][0]

    if 'submit-edition-rotation' in changed_id:
        df = utils.merge_df_str(str1, str2)
        return df.to_dict(orient='records')        
    else:
        return dash.no_update

# frame update - shape
@app.callback(
    Output(component_id='table-shape', component_property='data'),
    
    Input('submit-edition-shape', 'n_clicks'),
    Input("details-raport", "children"),
    Input("details-raport-edited-shape", "children"),
    prevent_initial_call=True,
)
def create_table_shape(n_clicks, str1, str2):
    changed_id = [p['prop_id'] for p in dash.callback_context.triggered][0]

    if 'submit-edition-shape' in changed_id:
        df = utils.merge_df_str(str1, str2)
        return df.to_dict(orient='records')        
    else:
        return dash.no_update

## PDF reports

In [ ]:
@app.callback(
    Output('hidden-div', 'children'),
    
    Input('report', 'n_clicks'),
    Input('graph-camera', 'figure'),#images
    Input('original-lime', 'figure'),
    Input('graph-lime', 'figure'),
    Input('graph-lime-edited', 'figure'),
    Input('graph-lime-edited-rotation', 'figure'),
    Input('graph-lime-edited-shape', 'figure'),
    
    Input("details-raport", "children"),#predictions
    Input("details-raport-edited", "children"),
    Input("details-raport-edited-rotation", "children"),
    Input("details-raport-edited-shape", "children"),
    
    Input("left-right", "value"),#parameters
    Input("up-down", "value"),
    Input("angle-input", "value"),
    Input("power-input", "value"),
    
    prevent_initial_call=True,  
)
def generate_report(n_clicks, fig1,fig2,fig3,fig4,fig5,fig6, str1,str2,str3,str4, horizontal,vertical,angle,power):
    changed_id = [p['prop_id'] for p in dash.callback_context.triggered][0]

    if 'report' in changed_id:
        #save images to file
        img_str1 = fig1["data"][0]["source"]
        img_str1 = img_str1.split(",")
        img1 = Image.open(BytesIO(base64.b64decode(img_str1[1])))
        img1 = np.array(img1)
        plt.imsave('img1.png', img1)
        img_str2 = fig2["data"][0]["source"]
        img_str2 = img_str2.split(",")
        img2 = Image.open(BytesIO(base64.b64decode(img_str2[1])))
        img2 = np.array(img2)
        plt.imsave('img2.png', img2)
        img_str3 = fig3["data"][0]["source"]
        img_str3 = img_str3.split(",")
        img3 = Image.open(BytesIO(base64.b64decode(img_str3[1])))
        img3 = np.array(img3)
        plt.imsave('img3.png', img3)
        img_str4 = fig4["data"][0]["source"]
        img_str4 = img_str4.split(",")
        img4 = Image.open(BytesIO(base64.b64decode(img_str4[1])))
        img4 = np.array(img4)
        plt.imsave('img4.png', img4)
        img_str5 = fig5["data"][0]["source"]
        img_str5 = img_str5.split(",")
        img5 = Image.open(BytesIO(base64.b64decode(img_str5[1])))
        img5 = np.array(img5)
        plt.imsave('img5.png', img5)
        img_str6 = fig6["data"][0]["source"]
        img_str6 = img_str6.split(",")
        img6 = Image.open(BytesIO(base64.b64decode(img_str6[1])))
        img6 = np.array(img6)
        plt.imsave('img6.png', img6)        
        
        #read images
        pdf = FPDF()
        pdf.add_page()
        pdf.set_font('Arial', 'B', 16)
        pdf.cell(40, 10, 'Report')
        w = 25
        h = 25
        space = 8

        #image titles
        pdf.set_font('Arial', size=8)
        pdf.set_xy(10, 35)
        pdf.cell(0, 0, 'original image')
        pdf.set_xy(10+(w+space), 35)
        pdf.cell(0, 0, 'original image + lime')
        pdf.set_xy(10+2*(w+space), 35)
        pdf.cell(0, 0, 'original image + limecraft')
        pdf.set_xy(10+3*(w+space), 35)
        pdf.cell(0, 0, 'edited colors + limecraft')
        pdf.set_xy(10+4*(w+space), 35)
        pdf.cell(0, 0, 'edited placement + limecraft')
        pdf.set_xy(10+5*(w+space), 35)
        pdf.cell(0, 0, 'edited shape + limecraft')

        #images
        pdf.image('img1.png', x = 10, y = 40, w = w, h = h) #original photo
        pdf.image('img2.png', x = 10+w+space, y = 40, w = w, h = h) #original lime
        pdf.image('img3.png', x = 10+2*(w+space), y = 40, w = w, h = h) #original limecraft
        pdf.image('img4.png', x = 10+3*(w+space), y = 40, w = w, h = h) #color
        pdf.image('img5.png', x = 10+4*(w+space), y = 40, w = w, h = h) #rotation and shift
        pdf.image('img6.png', x = 10+5*(w+space), y = 40, w = w, h = h) #shape

        #parameters
        pdf.set_font('Arial', size=6)
        pdf.set_xy(10+4*(w+space), 70) #rotation
        pdf.cell(0, 0, 'horizontal: {} px'.format(horizontal))
        pdf.set_xy(10+4*(w+space), 75)
        pdf.cell(0, 0, 'vertical: {} px'.format(vertical))
        pdf.set_xy(10+4*(w+space), 80)
        pdf.cell(0, 0, 'angle of rotation: {}'.format(angle))
        pdf.set_xy(10+5*(w+space), 70) #shape
        pdf.cell(0, 0, 'power of expansion: {}'.format(power))
        
        df = utils.merge_all_preds(str1,str2,str3,str4)
        data = list(df.itertuples(index=False))
        data.insert(0,list(df.columns))

        #table
        pdf.set_xy(10,90) #table position
        line_height = pdf.font_size * 2.5
        col_width = pdf.w/6
        for row in data:
            for datum in row:
                pdf.cell(col_width, line_height, str(datum), border=1)
            pdf.ln(line_height)
        

        pdf.output('report.pdf', 'F')
        
        return
    else:
        return dash.no_update

## Buttons

In [ ]:
# Disable button
@app.callback(
    Output('submit-val', 'disabled'),
    Input('submit-val', 'n_clicks'),
    Input('enable-button', 'n_clicks'),
)
def enable_button(n_clicks, n_clicks_enable):
    changed_id = [p['prop_id'] for p in dash.callback_context.triggered][0]
    if "submit-val" in changed_id:
        return True
    elif "enable-button" in changed_id:
        return False
    
# Automatic submission
@app.callback(
    Output('submit-uploading', 'n_clicks'),
    Input('upload-image', 'contents'),
)
def autoclick_button(contents):
    if contents is not None:
        return 1

In [ ]:
app.run_server(debug=False, host='127.0.0.1', port=8001)